In [1]:
!pip install pyelftools --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
obj_dir = Path("..").resolve() / "bcc" / "libbpf-tools" / ".output"

In [4]:
from utils import get_structs

get_structs(obj_dir / "bindsnoop.bpf.o")

/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.btf already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.h already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.txt already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.json already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.jsonl already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.pkl already exists


['in6_addr', 'sock', 'socket', 'sock_common', 'inet_sock']

In [5]:
from utils import get_hooks

get_hooks(obj_dir / "bindsnoop.bpf.o")

{('kprobe', 'inet6_bind'), ('kprobe', 'inet_bind')}

In [6]:
result = {}

for obj_file in sorted(obj_dir.glob("*.bpf.o")):
    hooks = get_hooks(obj_file)
    structs = get_structs(obj_file)
    result[obj_file.name] = (hooks, structs)

/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.btf already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.h already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.txt already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.json already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.jsonl already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.pkl already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.btf already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.h already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.txt already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.json already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bin

In [7]:
from utils import get_sorted_linux_paths
from pathlib import Path

paths = []
dirs = sorted(Path("data").glob("*-x86"))
for d in dirs:
    p = get_sorted_linux_paths(d.glob("*.pkl"))
    if d == dirs[-1]:
        paths.extend(p)
    else:
        paths.extend(p[:-1])

paths

[PosixPath('data/16.04-x86/4.4.0-210-generic.pkl'),
 PosixPath('data/16.04-x86/4.8.0-58-generic.pkl'),
 PosixPath('data/16.04-x86/4.10.0-42-generic.pkl'),
 PosixPath('data/16.04-x86/4.13.0-45-generic.pkl'),
 PosixPath('data/18.04-x86/4.15.0-213-generic.pkl'),
 PosixPath('data/18.04-x86/4.18.0-25-generic.pkl'),
 PosixPath('data/18.04-x86/5.0.0-65-generic.pkl'),
 PosixPath('data/18.04-x86/5.3.0-76-generic.pkl'),
 PosixPath('data/20.04-x86/5.4.0-170-generic.pkl'),
 PosixPath('data/20.04-x86/5.8.0-63-generic.pkl'),
 PosixPath('data/20.04-x86/5.11.0-46-generic.pkl'),
 PosixPath('data/20.04-x86/5.13.0-52-generic.pkl'),
 PosixPath('data/22.04-x86/5.15.0-97-generic.pkl'),
 PosixPath('data/22.04-x86/5.19.0-50-generic.pkl'),
 PosixPath('data/22.04-x86/6.2.0-39-generic.pkl'),
 PosixPath('data/22.04-x86/6.5.0-21-generic.pkl')]

In [8]:
from utils import BTF

btfs = [BTF(path) for path in paths]

In [9]:
from utils import Kind, get_changes

print(get_changes(btfs, Kind.STRUCT, "inet_sock"))
print(get_changes(btfs, Kind.FUNC, "blk_account_io_start"))

[('4.4', '4.8', [<StructChange.LAYOUT: 'Layout changed'>]), ('4.8', '4.10', [<StructChange.LAYOUT: 'Layout changed'>]), ('4.10', '4.13', [<StructChange.ADD: 'Field added'>, <StructChange.LAYOUT: 'Layout changed'>]), ('4.13', '4.15', [<StructChange.LAYOUT: 'Layout changed'>]), ('4.15', '4.18', [<StructChange.LAYOUT: 'Layout changed'>]), ('4.18', '5.0', [<StructChange.LAYOUT: 'Layout changed'>]), ('5.0', '5.3', [<StructChange.LAYOUT: 'Layout changed'>]), ('5.3', '5.4', [<StructChange.LAYOUT: 'Layout changed'>]), ('5.4', '5.8', [<StructChange.LAYOUT: 'Layout changed'>]), ('5.8', '5.11', [<StructChange.ADD: 'Field added'>, <StructChange.LAYOUT: 'Layout changed'>]), ('5.11', '5.13', [<StructChange.REMOVE: 'Field removed'>, <StructChange.LAYOUT: 'Layout changed'>]), ('5.13', '5.15', [<StructChange.LAYOUT: 'Layout changed'>]), ('5.15', '5.19', [<StructChange.LAYOUT: 'Layout changed'>]), ('5.19', '6.2', [<StructChange.ADD: 'Field added'>, <StructChange.TYPE: 'Field type changed'>, <StructChang

In [10]:
from utils import get_available_versions

print(get_available_versions(btfs, Kind.STRUCT, "renamedata"))
print(get_available_versions(btfs, Kind.FUNC, "do_page_cache_ra"))

['5.13', '5.15', '5.19', '6.2', '6.5']
['5.11', '5.13', '5.15']


In [23]:
from utils import StructChange

all_versions = [btf.short_version for btf in btfs]

def get_available_versions_str(available_versions):
    versions_str = "|"
    for v in all_versions:
        if v in available_versions:
            versions_str += "Y"
        else:
            versions_str += "N"
    percent = len(available_versions) / len(all_versions) * 100
    versions_str += f"| ({len(available_versions)}/{len(all_versions)} = {percent:.0f}%)"
    return versions_str
    
def print_changes(kind, name, t):
    print(f"\t{t}: {name}")
    if t not in ["kprobe", "tracepoint", "struct"]:
        return

    if t == "tracepoint":
        name = f"perf_trace_{name}"
        

    available_versions = get_available_versions(btfs, kind, name)
    print(f"\t\t Available: {get_available_versions_str(available_versions)}")

    changes = get_changes(btfs, kind, name)
    result = []
    for (v1, v2, reasons) in changes:
        desc = [f'{c.value}' for c in reasons if c != StructChange.LAYOUT]
        if desc:
            result.append(f"{v1} -> {v2}: {desc}")
    if not result:
        print(f"\t\t Changes: None")
    else:
        print(f"\t\t Changes ({len(result)}):")
        for r in result:
            print(f"\t\t\t {r}")
    
        
for name, (hooks, structs) in result.items():
    print(name)
    for (hook_type, hook_name) in hooks:
        print_changes(Kind.FUNC, hook_name, hook_type)
    for struct in structs:
        print_changes(Kind.STRUCT, struct, "struct")
    print()

bashreadline.bpf.o
	uprobe: readline

bindsnoop.bpf.o
	kprobe: inet6_bind
		 Available: |YYYYYYYYYYYYYYYY| (16/16 = 100%)
		 Changes: None
	kprobe: inet_bind
		 Available: |YYYYYYYYYYYYYYYY| (16/16 = 100%)
		 Changes: None
	struct: in6_addr
		 Available: |YYYYYYYYYYYYYYYY| (16/16 = 100%)
		 Changes: None
	struct: sock
		 Available: |YYYYYYYYYYYYYYYY| (16/16 = 100%)
		 Changes (14):
			 4.4 -> 4.8: ['Field added', 'Field removed', 'Field type changed']
			 4.8 -> 4.10: ['Field added']
			 4.10 -> 4.13: ['Field added', 'Field type changed']
			 4.13 -> 4.15: ['Field added', 'Field removed']
			 4.15 -> 4.18: ['Field added', 'Field removed']
			 4.18 -> 5.0: ['Field added', 'Field type changed']
			 5.0 -> 5.3: ['Field added']
			 5.3 -> 5.4: ['Field added']
			 5.4 -> 5.8: ['Field removed', 'Field type changed']
			 5.8 -> 5.11: ['Field added', 'Field type changed']
			 5.11 -> 5.13: ['Field added']
			 5.13 -> 5.15: ['Field added', 'Field type changed']
			 5.15 -> 5.19: ['Field added',